In [1]:
%pip install gensim
%pip install nltk
%pip install pymoo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('../dataset/jiradataset_issues_v1.4.csv', encoding='utf-8')

In [3]:
dataset.head()

,assignee.name,components,created,creator.name,description,fixVersions,issuetype.name,issuetype.subtask,priority.name,reporter.name,...,status.name,status.statusCategory.name,summary,updated,versions,watches.watchCount,key,storypoints,project,sprint
0,NaN,"[{u'id': u'12786', u'self': u'https://jira.spr...",2016-03-31T22:35:55.000+0000,thomas.risberg,"The MapReduce samples should have ""yarn.resour...",[],Improvement,False,Minor,thomas.risberg,...,To Do,To Do,"Add ""yarn.resourcemanager.scheduler.address"" t...",2016-03-31T22:35:55.000+0000,[],1,XD-3753,2.0,xd,NaN
1,NaN,"[{u'id': u'12784', u'self': u'https://jira.spr...",2016-03-14T18:09:51.000+0000,manju4ever,When i use the admin-ui web portal which runs ...,[],Bug,False,Trivial,manju4ever,...,To Do,To Do,Admin UI login Page failing to load due to req...,2016-03-14T18:09:51.000+0000,"[{u'archived': False, u'description': u'1.3.0 ...",1,XD-3752,20.0,xd,NaN
2,jvalkeal,[],2016-03-03T13:22:14.000+0000,jvalkeal,In a case where reactor's ringbuffer is full a...,[],Bug,False,Major,jvalkeal,...,In PR,In Progress,gpfdist may fail to shutdown with backlog,2016-03-03T18:41:19.000+0000,"[{u'archived': False, u'name': u'1.3.1', u'sel...",1,XD-3751,5.0,xd,NaN
3,NaN,[],2016-02-29T10:00:18.000+0000,aliiqbal,if we put a custom module static properties in...,[],Bug,False,Major,aliiqbal,...,To Do,To Do,Cant completely remove custom module after put...,2016-03-13T10:24:15.000+0000,"[{u'archived': False, u'name': u'1.2.1', u'sel...",1,XD-3750,1.0,xd,NaN
4,NaN,[],2016-02-26T15:57:18.000+0000,grussell,Custom conversion is broken.\r\r\n\r\r\nIf the...,[],Bug,False,Major,grussell,...,To Do,To Do,Provisioning of Custom input/output Converters...,2016-02-26T16:08:03.000+0000,[],1,XD-3749,5.0,xd,NaN


In [4]:
dataset.describe()

,status.id,watches.watchCount,storypoints
count,111048.000000,111048.000000,2.484000e+04
mean,961.120479,2.452867,1.080227e+03
std,2944.655296,3.641834,1.694728e+05
min,1.000000,0.000000,0.000000e+00
25%,6.000000,1.000000,2.000000e+00
50%,6.000000,2.000000,3.000000e+00
75%,6.000000,3.000000,5.000000e+00
max,10953.000000,156.000000,2.671011e+07


Include only xd project

In [5]:
dataset = dataset.loc[dataset["project"] == "xd"]
dataset.describe()

,status.id,watches.watchCount,storypoints
count,3784.000000,3784.000000,3784.000000
mean,9945.356501,1.566332,4.321406
std,742.829822,0.835909,41.899467
min,3.000000,0.000000,0.000000
25%,10001.000000,1.000000,1.000000
50%,10001.000000,1.000000,3.000000
75%,10001.000000,2.000000,5.000000
max,10006.000000,10.000000,2567.000000


## Data prep

Merge description and summary into one column - text

In [6]:
dataset['description'] = dataset['description'].fillna("")
dataset['summary'] = dataset['summary'].fillna("")

dataset["text"] = dataset["description"] + " " + dataset["summary"]
dataset.columns.values

array(['assignee.name', 'components', 'created', 'creator.name',
       'description', 'fixVersions', 'issuetype.name',
       'issuetype.subtask', 'priority.name', 'reporter.name',
       'resolution.description', 'resolution.name', 'resolutiondate',
       'status.id', 'status.name', 'status.statusCategory.name',
       'summary', 'updated', 'versions', 'watches.watchCount', 'key',
       'storypoints', 'project', 'sprint', 'text'], dtype=object)

Map priorities to numerical values

In [7]:
import helpers.prioritymapper as pm

dataset["priority"]= dataset.apply(lambda x: pm.mapPriority(x["project"], x["priority.name"]), axis=1)
dataset.head()

,assignee.name,components,created,creator.name,description,fixVersions,issuetype.name,issuetype.subtask,priority.name,reporter.name,...,summary,updated,versions,watches.watchCount,key,storypoints,project,sprint,text,priority
0,NaN,"[{u'id': u'12786', u'self': u'https://jira.spr...",2016-03-31T22:35:55.000+0000,thomas.risberg,"The MapReduce samples should have ""yarn.resour...",[],Improvement,False,Minor,thomas.risberg,...,"Add ""yarn.resourcemanager.scheduler.address"" t...",2016-03-31T22:35:55.000+0000,[],1,XD-3753,2.0,xd,NaN,"The MapReduce samples should have ""yarn.resour...",2
1,NaN,"[{u'id': u'12784', u'self': u'https://jira.spr...",2016-03-14T18:09:51.000+0000,manju4ever,When i use the admin-ui web portal which runs ...,[],Bug,False,Trivial,manju4ever,...,Admin UI login Page failing to load due to req...,2016-03-14T18:09:51.000+0000,"[{u'archived': False, u'description': u'1.3.0 ...",1,XD-3752,20.0,xd,NaN,When i use the admin-ui web portal which runs ...,1
2,jvalkeal,[],2016-03-03T13:22:14.000+0000,jvalkeal,In a case where reactor's ringbuffer is full a...,[],Bug,False,Major,jvalkeal,...,gpfdist may fail to shutdown with backlog,2016-03-03T18:41:19.000+0000,"[{u'archived': False, u'name': u'1.3.1', u'sel...",1,XD-3751,5.0,xd,NaN,In a case where reactor's ringbuffer is full a...,3
3,NaN,[],2016-02-29T10:00:18.000+0000,aliiqbal,if we put a custom module static properties in...,[],Bug,False,Major,aliiqbal,...,Cant completely remove custom module after put...,2016-03-13T10:24:15.000+0000,"[{u'archived': False, u'name': u'1.2.1', u'sel...",1,XD-3750,1.0,xd,NaN,if we put a custom module static properties in...,3
4,NaN,[],2016-02-26T15:57:18.000+0000,grussell,Custom conversion is broken.\r\r\n\r\r\nIf the...,[],Bug,False,Major,grussell,...,Provisioning of Custom input/output Converters...,2016-02-26T16:08:03.000+0000,[],1,XD-3749,5.0,xd,NaN,Custom conversion is broken.\r\r\n\r\r\nIf the...,3


Add 1 story point for issues that don't have storypoints defined

In [8]:
dataset['storypoints'] = dataset['storypoints'].fillna(1)

Leave only necessary columns into the dataframe

In [9]:
dataset = dataset[["created", "assignee.name", "priority", "status.name", "text", "storypoints", "project", "resolutiondate", "description", "summary", "key"]]

Define backlog

In [10]:
backlog = dataset.loc[(dataset["assignee.name"].isna()) & (dataset["resolutiondate"].isna())]
backlog.describe()

,priority,storypoints
count,508.000000,508.000000
mean,2.120079,4.133858
std,0.375994,5.555334
min,1.000000,0.000000
25%,2.000000,1.000000
50%,2.000000,3.000000
75%,2.000000,5.000000
max,5.000000,100.000000


Define done issues

In [11]:
df = dataset.loc[(dataset["assignee.name"].notna()) & (dataset["resolutiondate"].notna())]
df.head()

,created,assignee.name,priority,status.name,text,storypoints,project,resolutiondate,description,summary,key
7,2016-02-23T18:18:48.000+0000,grussell,2,Done,Update Spring Framework to 4.2.4,1.0,xd,2016-02-23T18:19:06.000+0000,,Update Spring Framework to 4.2.4,XD-3746
8,2016-02-23T18:04:09.000+0000,grussell,2,Done,Update to amqp-client 3.6.0 and spring-amqp 1....,1.0,xd,2016-02-23T18:04:41.000+0000,Update to amqp-client 3.6.0 and spring-amqp 1.5.4,"Update Spring-AMQP to 3.6, RabbitMQ Client to ...",XD-3745
9,2016-02-22T15:27:31.000+0000,grussell,2,Done,"Related to XD-2567 which fixed this problem, b...",1.0,xd,2016-02-22T18:41:31.000+0000,"Related to XD-2567 which fixed this problem, b...",Suppress DeliveryMode Header in RabbitMQ Source,XD-3744
10,2016-02-17T14:28:31.000+0000,abilan,2,Done,See INT-3956 Update to Spring Integration 4.2....,1.0,xd,2016-02-21T02:28:25.000+0000,See INT-3956,Update to Spring Integration 4.2.5 When Availa...,XD-3743
11,2016-02-16T14:59:06.000+0000,dturanski,2,Done,The following XD components have been identifi...,3.0,xd,2016-02-19T20:04:19.000+0000,The following XD components have been identifi...,Enable in line SSL properties as an alternativ...,XD-3742


## LDA

Import necessary things for LDA

In [12]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sande\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Define preprocessing functions

In [13]:
# past and future tenses to present, third person to first person
def lemmatize_stemming(text):
    return SnowballStemmer("english").stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# remove stopwords and words that have 3 or less characters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

Example of preprocessing

In [14]:
preprocess("This is an example of a preprocessed issue text. who is thins?")

['exampl', 'preprocess', 'issu', 'text', 'thin']

Preprocess all issue texts

In [15]:
processed_docs = df['text'].map(str).map(preprocess)
processed_docs.head()

7                            [updat, spring, framework]
8     [updat, amqp, client, spring, amqp, updat, spr...
9     [relat, fix, problem, quot, releas, warn, supp...
10               [updat, spring, integr, avail, metric]
11    [follow, compon, identifi, support, sslpropert...
Name: text, dtype: object

In [16]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [17]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [18]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

Train the LDA model

In [19]:
number_of_topics = 3
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=number_of_topics, id2word=dictionary, passes=2, workers=2)

In [20]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.098*"java" + 0.057*"spring" + 0.033*"springframework" + 0.025*"modul" + 0.019*"stream" + 0.017*"user" + 0.016*"build" + 0.015*"document" + 0.014*"apach" + 0.012*"integr"
Topic: 1 
Words: 0.081*"java" + 0.068*"springframework" + 0.021*"integr" + 0.016*"hadoop" + 0.016*"spring" + 0.014*"support" + 0.011*"apach" + 0.011*"bean" + 0.011*"redi" + 0.011*"deploy"
Topic: 2 
Words: 0.033*"modul" + 0.025*"file" + 0.021*"spring" + 0.019*"creat" + 0.016*"stream" + 0.014*"java" + 0.013*"springframework" + 0.013*"user" + 0.013*"support" + 0.012*"test"


Print the vector of the example issue

In [21]:
example_issue = df["text"].iloc[0]
print(example_issue)
bow_vector = dictionary.doc2bow(preprocess(example_issue))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))


 Update Spring Framework to 4.2.4
Score: 0.7923011183738708	 Topic: 0.098*"java" + 0.057*"spring" + 0.033*"springframework" + 0.025*"modul" + 0.019*"stream"
Score: 0.10753729194402695	 Topic: 0.081*"java" + 0.068*"springframework" + 0.021*"integr" + 0.016*"hadoop" + 0.016*"spring"
Score: 0.10016155242919922	 Topic: 0.033*"modul" + 0.025*"file" + 0.021*"spring" + 0.019*"creat" + 0.016*"stream"


## Add topic vectors to backlog issues

In [22]:
def get_topic_vector(text, lda):
    bow_vector = dictionary.doc2bow(preprocess(text))
    topic_vector = lda[bow_vector]
    return [x for x in topic_vector]


Convert from the LDA output to an actual vector

In [23]:
def topic_to_vector(topic, number_of_topics):
    vector = np.zeros(number_of_topics)
    for x in topic:
        vector[x[0]] = x[1]
    return vector

In [24]:
get_topic_vector(example_issue, lda_model)

[(0, 0.79231143), (1, 0.10752494), (2, 0.10016363)]

Add topic vector to all backlog issues

In [25]:
backlog["vector"] = backlog.apply(lambda x: get_topic_vector(x["text"], lda_model), axis=1)
backlog.head()

C:\Users\sande\AppData\Local\Temp/ipykernel_8012/2122134350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backlog["vector"] = backlog.apply(lambda x: get_topic_vector(x["text"], lda_model), axis=1)


,created,assignee.name,priority,status.name,text,storypoints,project,resolutiondate,description,summary,key,vector
0,2016-03-31T22:35:55.000+0000,NaN,2,To Do,"The MapReduce samples should have ""yarn.resour...",2.0,xd,NaN,"The MapReduce samples should have ""yarn.resour...","Add ""yarn.resourcemanager.scheduler.address"" t...",XD-3753,"[(0, 0.03448248), (1, 0.7311464), (2, 0.234371..."
1,2016-03-14T18:09:51.000+0000,NaN,1,To Do,When i use the admin-ui web portal which runs ...,20.0,xd,NaN,When i use the admin-ui web portal which runs ...,Admin UI login Page failing to load due to req...,XD-3752,"[(0, 0.01960501), (1, 0.40486413), (2, 0.57553..."
3,2016-02-29T10:00:18.000+0000,NaN,3,To Do,if we put a custom module static properties in...,1.0,xd,NaN,if we put a custom module static properties in...,Cant completely remove custom module after put...,XD-3750,"[(2, 0.98503727)]"
4,2016-02-26T15:57:18.000+0000,NaN,3,To Do,Custom conversion is broken.\r\r\n\r\r\nIf the...,5.0,xd,NaN,Custom conversion is broken.\r\r\n\r\r\nIf the...,Provisioning of Custom input/output Converters...,XD-3749,"[(0, 0.010595654), (1, 0.086828455), (2, 0.902..."
6,2016-02-23T18:31:37.000+0000,NaN,2,To Do,http://stackoverflow.com/questions/35563064/pr...,1.0,xd,NaN,http://stackoverflow.com/questions/35563064/pr...,Rabbit Bus: Expose ChannelCacheSize on Caching...,XD-3747,"[(0, 0.04806215), (1, 0.23565559), (2, 0.71628..."


In [26]:
backlog["vector2"] = backlog["vector"].apply(topic_to_vector, args=(number_of_topics,))
backlog.head()

C:\Users\sande\AppData\Local\Temp/ipykernel_8012/1151676804.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  backlog["vector2"] = backlog["vector"].apply(topic_to_vector, args=(number_of_topics,))


,created,assignee.name,priority,status.name,text,storypoints,project,resolutiondate,description,summary,key,vector,vector2
0,2016-03-31T22:35:55.000+0000,NaN,2,To Do,"The MapReduce samples should have ""yarn.resour...",2.0,xd,NaN,"The MapReduce samples should have ""yarn.resour...","Add ""yarn.resourcemanager.scheduler.address"" t...",XD-3753,"[(0, 0.03448248), (1, 0.7311464), (2, 0.234371...","[0.034482479095458984, 0.7311463952064514, 0.2..."
1,2016-03-14T18:09:51.000+0000,NaN,1,To Do,When i use the admin-ui web portal which runs ...,20.0,xd,NaN,When i use the admin-ui web portal which runs ...,Admin UI login Page failing to load due to req...,XD-3752,"[(0, 0.01960501), (1, 0.40486413), (2, 0.57553...","[0.019605010747909546, 0.4048641324043274, 0.5..."
3,2016-02-29T10:00:18.000+0000,NaN,3,To Do,if we put a custom module static properties in...,1.0,xd,NaN,if we put a custom module static properties in...,Cant completely remove custom module after put...,XD-3750,"[(2, 0.98503727)]","[0.0, 0.0, 0.9850372672080994]"
4,2016-02-26T15:57:18.000+0000,NaN,3,To Do,Custom conversion is broken.\r\r\n\r\r\nIf the...,5.0,xd,NaN,Custom conversion is broken.\r\r\n\r\r\nIf the...,Provisioning of Custom input/output Converters...,XD-3749,"[(0, 0.010595654), (1, 0.086828455), (2, 0.902...","[0.010595654137432575, 0.08682845532894135, 0...."
6,2016-02-23T18:31:37.000+0000,NaN,2,To Do,http://stackoverflow.com/questions/35563064/pr...,1.0,xd,NaN,http://stackoverflow.com/questions/35563064/pr...,Rabbit Bus: Expose ChannelCacheSize on Caching...,XD-3747,"[(0, 0.04806215), (1, 0.23565559), (2, 0.71628...","[0.048062149435281754, 0.23565559089183807, 0...."


## Add "experience" topic vectors to users

Concatenate all issue texts for users

In [27]:
user_df = df.groupby("assignee.name").agg({'text': 'sum'})
user_df["text"].iloc[0]

"See INT-3956 Update to Spring Integration 4.2.5 When Available (Fix Metrics)As a user, I'd like to have the option of _Cassandra_ sink, so I can leverage the NoSQL database to write high volumes of variable data segments in high velocity. Add Cassandra sink"

In [28]:
user_df["vector"] = user_df["text"].apply(get_topic_vector, args=(lda_model,))
user_df.head()

,text,vector
assignee.name,,
abilan,See INT-3956 Update to Spring Integration 4.2....,"[(0, 0.02386925), (1, 0.02406439), (2, 0.95206..."
aclement,"As a developer, I'd like to enhance test cover...","[(2, 0.9989938)]"
aeisenberg,We should consider moving to wire.js to encour...,"[(1, 0.08098623), (2, 0.9157689)]"
chrisschaefer,"Based on the the results from B-6, select the ...","[(2, 0.9967869)]"
david_syer,Use Boot plugin and IO Platform for versions ...,"[(1, 0.53642166), (2, 0.4573191)]"


In [29]:
user_df["vector2"] = user_df["vector"].apply(topic_to_vector, args=(number_of_topics,))
user_df.head()

,text,vector,vector2
assignee.name,,,
abilan,See INT-3956 Update to Spring Integration 4.2....,"[(0, 0.02386925), (1, 0.02406439), (2, 0.95206...","[0.02386924996972084, 0.02406438998878002, 0.9..."
aclement,"As a developer, I'd like to enhance test cover...","[(2, 0.9989938)]","[0.0, 0.0, 0.9989938139915466]"
aeisenberg,We should consider moving to wire.js to encour...,"[(1, 0.08098623), (2, 0.9157689)]","[0.0, 0.08098623156547546, 0.9157689213752747]"
chrisschaefer,"Based on the the results from B-6, select the ...","[(2, 0.9967869)]","[0.0, 0.0, 0.996786892414093]"
david_syer,Use Boot plugin and IO Platform for versions ...,"[(1, 0.53642166), (2, 0.4573191)]","[0.0, 0.5364216566085815, 0.45731911063194275]"


In [30]:
from scipy import spatial
import functools

similarity_to_experience = functools.partial(spatial.distance.cosine, user_df["vector2"].iloc[3])

backlog[["key", "vector2", "storypoints", "priority"]].to_numpy()

array([['XD-3753', array([0.03448248, 0.7311464 , 0.23437116]), 2.0, 2],
       ['XD-3752', array([0.01960501, 0.40486413, 0.57553089]), 20.0, 1],
       ['XD-3750', array([0.        , 0.        , 0.98503727]), 1.0, 3],
       ...,
       ['XD-801', array([0.6990335 , 0.02443375, 0.27653277]), 1.0, 2],
       ['XD-387', array([0.24646075, 0.0624222 , 0.69111705]), 0.0, 3],
       ['XD-346', array([0.01929029, 0.09465125, 0.88605851]), 1.0, 2]],
      dtype=object)